In [3]:
%cd "/content/drive/MyDrive/SkinFormer"

/content/drive/MyDrive/SkinFormer


In [5]:
!pip install -q einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.8 MB/s eta 0:00:00


In [6]:
!python Train_DeiT-B_LRP.py --config configs.yml

CUDA is available: True 

Starting... 

{'train': 12809, 'val': 3203}
Downloading: "https://dl.fbaipublicfiles.com/deit/deit_base_patch16_224-b5f2ef4d.pth" to /root/.cache/torch/hub/checkpoints/deit_base_patch16_224-b5f2ef4d.pth
100% 330M/330M [00:04<00:00, 82.5MB/s]
Traceback (most recent call last):
  File "/content/drive/MyDrive/SkinFormer/Train_DeiT-B_LRP.py", line 292, in <module>
    main(config)
  File "/content/drive/MyDrive/SkinFormer/Train_DeiT-B_LRP.py", line 227, in main
    model = deit_base_patch16_224(
  File "/content/drive/MyDrive/SkinFormer/Models/ViT_LRP/ViT_LRP.py", line 536, in deit_base_patch16_224
    model.load_state_dict(checkpoint["model"])
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 2041, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for VisionTransformer:
	size mismatch for head.weight: copying a param with shape torch.Size([1